<a href="https://colab.research.google.com/github/andrewmk/d3-examples/blob/main/d3_magic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create d3 magic

In [2]:
from urllib.parse import quote
from IPython.core.magic import register_cell_magic
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring
from IPython.display import HTML


@magic_arguments()
@argument('-h', '--height', default='350px', help='Height of the output area')
@argument('-w', '--width', default='100%', help='Width of the output area')
@argument('-s', '--style', default='border: none', help='Style of the output area')
@argument('-v', '--version', default='7.2.1', help='D3.js version')
@argument('-i', '--integrity', default=None, help='Integrity hash for d3.js; when changing version provide one for security')
@argument('-c', '--cdn', default='https://cdnjs.cloudflare.com/ajax/libs/d3', help='CDN address')
@register_cell_magic
def d3(line, cell):
    args = parse_argstring(d3, line)
    known_versions = {
        '4.13.0': 'sha512-RJJ1NNC88QhN7dwpCY8rm/6OxI+YdQP48DrLGe/eSAd+n+s1PXwQkkpzzAgoJe4cZFW2GALQoxox61gSY2yQfg==',
        '7.2.1':  'sha512-wkduu4oQG74ySorPiSRStC0Zl8rQfjr/Ty6dMvYTmjZw6RS5bferdx8TR7ynxeh79ySEp/benIFFisKofMjPbg==' 
    }
    if not args.integrity:
        args.integrity = known_versions.get(args.version, '')
        
    integrity = 'integrity="{}"'.format(args.integrity) if args.integrity else ''

    content = quote(
        """
        <html>
            <body>
            <script src='{args.cdn}/{args.version}/d3.min.js' {integrity} crossorigin="anonymous"></script>
            <script src='https://d3js.org/d3-selection-multi.v1.min.js' crossorigin="anonymous"></script>
            <script>document.body.onload = function () {{ {code} }}</script>
            </body>
        </html>
        """.format(code=cell, args=args, integrity=integrity),
        safe=''
    )
    html = """
    <iframe
        style="{args.style}"
        width="{args.width}"
        height="{args.height}"
        sandbox="allow-scripts allow-modals"
        referrerpolicy="no-referrer"
        src="data:text/html;charset=UTF-8,{content}"
    ></iframe>
    """.format(content=content, args=args)
    return HTML(html)

# Basics

## Check version being used

In [ ]:
%%d3 -h 30
d3.select("body").text(d3.version)

## Hello World

In [ ]:
%%d3 -h 40
d3.select('body')
 .append('h1')
 .text('Hello World!');

## Animated circle test

In [ ]:
%%d3 -s "border: 1"
var svg = d3.select("body")
    .append("svg")
    .attr("width", 300)
    .attr("height", 300);    

svg.append("circle")
    .style("stroke", "gray")
    .style("fill", "cyan")
    .attr("r", 130)
    .attr("cx", 150)
    .attr("cy", 150)
    .transition()
        .delay(100)
        .duration(10000)  
        .attr("r", 10)
        .attr("cx", 150)
        .style("fill", "blue"); 

## Test data binding

In [ ]:
%%d3 -v 7.2.1 -h 80
var selector = d3.select('body')
 .selectAll('div')
 .data([10, 20, 30, 40]);

var entering = selector.enter();

entering.append('div').text(function(d) { return 'hello ' + d; });


## Test updating values

In [ ]:
%%d3 -v 7.2.1 -h 180
function log(msg) {
    d3.select('body').select('div#log').append('p').text(msg);
}

function setupHtml() {
    d3.select('body').append('div').attr('id', 'charts');
    var button = d3.select('body').append('button').text('Modify').on('click', modify, 'onclick');
    d3.select('body').append('div').attr('id', 'log');   
}

function modify() {
    render([20, 30, 50], 'onclick');
}

function render(dataToRender, msg='') {
    log(`in render function: ${msg}`);
    var selector = d3.select('body').select('div#charts').selectAll('div').data(dataToRender);
    var entering = selector.enter();
    log(`selector: ${selector.size()}, entering: ${entering.size()}`);
    entering.append('div').text(function(d) { return d; });
    
    selector.text(function(d) { return d; });
}

setupHtml();
render([10, 20, 30], 'first');

## Test removing values

In [ ]:
%%d3 -h 220
function log(msg) {
    d3.select('body').select('div#log').append('p').text(msg);
}

function setupHtml() {
    html=`
        <div id='charts'>
        </div>
        <div id='log'>
        </div>
    `;
    d3.select('body').append('button').text('Modify').on('click', modify, 'onclick');
    d3.select('body').append('div').html(html);
}

function modify() {
    render([5, 15], 'onclick');
}

function render(dataToRender, msg='') {
    log(`in render function: ${msg}`);
    var selector = d3.select('body').select('div#charts').selectAll('div').data(dataToRender);
    var entering = selector.enter();
    var exiting = selector.exit();
    log(`selector: ${selector.size()}, entering: ${entering.size()}, exiting: ${exiting.size()}`);
    
    entering.append('div').text(function(d) { return d; });
    
    selector.text(function(d) { return d; });
    
    exiting.remove();
}

setupHtml();
render([10, 20, 30], 'first');

# SVG

In [ ]:
%%d3 -h 140 -s "border"

function setupHtml() {
    html=`
        <svg width="720" height="120">
            <circle cx="40" cy="20" r="10" style="fill:red"></circle>
            <circle cx="80" cy="40" r="15" style="fill:green"></circle>
            <circle cx="120" cy="60" r="20" style="fill:blue"></circle>
        </svg>
    `;
    d3.select('body').append('div').html(html);
}

setupHtml();

In [ ]:
%%d3 -h 140 -s "border"

function setupHtml() {
    html=`
        <svg width="720" height="120">
            <circle cx="40" cy="20" r="10" style="fill:red"></circle>
            <circle cx="80" cy="40" r="15" style="fill:green"></circle>
            <circle cx="120" cy="60" r="20" style="fill:blue"></circle>
        </svg>
    `;
    d3.select('body').append('div').html(html);
}

setupHtml();
d3.select('body').selectAll('circle').style('fill', 'teal');

In [ ]:
%%d3 -h 140 -s "border"

function setupHtml() {
    html=`
        <svg width="720" height="120">
            <line x1="10" y1="20" x2="110" y2="100"
             stroke="red" stroke-width="20" stroke-linecap="butt" />
            <line x1="60" y1="20" x2="160" y2="100"
             stroke="green" stroke-width="20" stroke-linecap="square" />
            <line x1="110" y1="20" x2="210" y2="100"
             stroke="blue" stroke-width="20" stroke-linecap="round" />
            <path d="M 10 20 L 110 100 M 60 20 L 160 100 M 110 20 L 210 100"
             stroke="white" />
        </svg>
    `;
    d3.select('body').append('div').html(html);
}

setupHtml();

In [ ]:
%%d3 -h 180 -s "border"

function setupHtml() {
    html=`
        <svg width="720" height="160">
            <rect x="0" y="0" width="100" height="100" fill="red"
                 transform="translate(30,30) rotate(45,50,50)" />
        </svg>
    `;
    d3.select('body').append('div').html(html);
}

setupHtml();

In [ ]:
%%d3 -h 180 -s "border"

function setupHtml() {
    html=`
        <svg>
            <rect x="0" y="0" width="50" height="50" fill="green"/>
            <g transform="translate(100,30) rotate(45 50 50)">
                 <rect x="0" y="0" width="100" height="100" style="fill:blue" />
                 <text x="15" y="58" fill="White" font-family="arial" font-size="16">
                     In the box
                 </text>
            </g>
        </svg>
    `;
    d3.select('body').append('div').html(html);
}

setupHtml();

# Bar graphs

In [ ]:
%%d3 -s border -h 226 -w 516

var data = [55, 44, 30, 23, 17, 14, 16, 25, 41, 61, 85,
 101, 95, 105, 114, 150, 180, 210, 125, 100, 71,
 75, 72, 67];

var barWidth = 15, barPadding = 3;
var maxValue = d3.max(data);

var svg = d3.select('body').append('svg').attr('width', 500).attr('height', 210);
var barGroup = svg.append('g');

function xloc(d, i) { return i * (barWidth + barPadding); }
function yloc(d) { return maxValue - d; }
function translator(d, i) {
    return "translate(" + xloc(d, i) + "," + yloc(d) + ")";
}

barGroup.selectAll("rect")
    .data(data)
    .enter()
    .append('rect')
    .attr('fill','steelblue')
    .attr('transform', translator)
    .attr('width', barWidth)
    .attr('height', function (d) { return d; });

In [ ]:
%%d3 -s border -h 226 -w 516

var data = [55, 44, 30, 23, 17, 14, 16, 25, 41, 61, 85,
 101, 95, 105, 114, 150, 180, 210, 125, 100, 71,
 75, 72, 67];

var barWidth = 15, barPadding = 3;
var maxValue = d3.max(data);

var svg = d3.select('body').append('svg').attr('width', 500).attr('height', 210);
var barGroup = svg.append('g');

function xloc(d, i) { return i * (barWidth + barPadding); }
function yloc(d) { return maxValue - d; }
function translator(d, i) {
    return "translate(" + xloc(d, i) + "," + yloc(d) + ")";
}

barGroup.selectAll("rect")
    .data(data)
    .enter()
    .append('rect')
    .attrs({
        'fill': 'steelblue',
        'transform': translator,
        'width': barWidth,
        'height': function (d) { return d; }
    });

In [ ]:
%%d3 -s border -h 226 -w 1016

var data = [55, 44, 30, 23, 17, 14, 16, 25, 41, 61, 85,
 101, 95, 105, 114, 150, 180, 210, 125, 100, 71,
 75, 72, 67];

var barWidth = 20, barPadding = 3;
var maxValue = d3.max(data);

var svg = d3.select('body').append('svg').attr('width', 1000).attr('height', 210);
var chart = svg.append('g');

function xloc(d, i) { return i * (barWidth + barPadding); }
function yloc(d) { return maxValue - d; }
function translator(d, i) {
    return "translate(" + xloc(d, i) + "," + yloc(d) + ")";
}

var barGroups = chart.selectAll("rect")
    .data(data)
    .enter()
    .append('g')
    .attr('transform', translator);
    
barGroups.append('rect')
    .attrs({
        'fill': 'steelblue',
        'width': barWidth,
        'height': 0
    })
    .transition()
        .delay(100)
        .duration(1000)  
        .attr('height', function(d) { return d; });

var textTranslator = "translate(" + barWidth / 2 + ", 2)";

barGroups.append('text')
    .attrs({
        'transform': textTranslator,
        'fill': 'white',
        'alignment-baseline': 'before-edge',
        'text-anchor': 'middle',  
    })
    .style('font', '10px sans-serif')
    .text(function (d) { return d; });

In [ ]:
%%d3 -h 300 -w 1100

var data = [55, 44, 30, 23, 17, 14, 16, 25, 41, 61, 85,
 101, 95, 105, 114, 150, 180, 210, 125, 100, 71,
 75, 72, 67];

var barWidth = 20, barPadding = 3;
var maxValue = d3.max(data);

var graphWidth = data.length * (barWidth + barPadding) - barPadding;
var margin = { top: 10, right: 10, bottom: 10, left: 50 };
var totalWidth = graphWidth + margin.left + margin.right;
var totalHeight = maxValue + margin.top + margin.bottom;

var svg = d3.select('body').append('svg').attrs({
    'width': totalWidth,
    'height': totalHeight
});

svg.append('rect').attrs({
    width: totalWidth,
    height: totalHeight,
    fill: 'white',
    stroke: 'black',
    'stroke-width': 1
});

var chart = svg.append('g').attr('transform', 'translate(' + margin.left + ',' + margin.top + ")");
chart.append('rect').attrs({
    fill: 'rgba(0,0,0,0.1)',
    width: totalWidth - (margin.left + margin.right),
    height: totalHeight - (margin.bottom + margin.top)
});

function xloc(d, i) { return i * (barWidth + barPadding); }
function yloc(d) { return maxValue - d; }
function translator(d, i) {
    return "translate(" + xloc(d, i) + "," + yloc(d) + ")";
}

var barGroups = chart.selectAll("g")
    .data(data)
    .enter()
    .append('g')
    .attrs({'transform': translator});
    
barGroups.append('rect')
    .attrs({
        'fill': 'steelblue',
        'width': barWidth,
        'height': function(d) { return d; }
    });

var textTranslator = "translate(" + barWidth / 2 + ", 2)";

barGroups.append('text')
    .attrs({
        'transform': textTranslator,
        'fill': 'white',
        'alignment-baseline': 'before-edge',
        'text-anchor': 'middle',  
    })
    .style('font', '10px sans-serif')
    .text(function (d) { return d; });

In [ ]:
%%d3 -h 50 -s border
var data = [55, 44, 30, 23, 17, 14, 16, 25, 41, 61, 85,
                    101, 95, 105, 114, 150, 180, 210, 125, 100, 71,
                    75, 72, 67];

        var maxValue = d3.max(data);

        var width = 500, height = 300;
        var svg = d3.select('body')
            .append('svg')
            .attrs({ width: width, height: height });

        var scale = d3.scaleLinear()
            .domain([0, maxValue])
            .range([0, width]);

        var axisGroup = svg.append('g');
        var axis = d3.axisBottom().scale(scale);
        axisGroup.call(axis).attr('transform', 'translate(0,0)');

In [ ]:
%%d3 -h 280 -w 1100

var data = [55, 44, 30, 23, 17, 14, 16, 25, 41, 61, 85,
 101, 95, 105, 114, 150, 180, 210, 125, 100, 71,
 75, 72, 67];

var barWidth = 20, barPadding = 3;
var maxValue = d3.max(data);

var graphWidth = data.length * (barWidth + barPadding) - barPadding;
var margin = { top: 10, right: 10, bottom: 10, left: 50 };
var totalWidth = graphWidth + margin.left + margin.right;
var totalHeight = maxValue + margin.top + margin.bottom;

var svg = d3.select('body').append('svg').attrs({
    'width': totalWidth,
    'height': totalHeight
});

svg.append('rect').attrs({
    width: totalWidth,
    height: totalHeight,
    fill: 'white',
    stroke: 'black',
    'stroke-width': 1
});

var chart = svg.append('g').attr('transform', 'translate(' + margin.left + ',' + margin.top + ")");
chart.append('rect').attrs({
    fill: 'rgba(0,0,0,0.5)',
    width: totalWidth - (margin.left + margin.right),
    height: totalHeight - (margin.bottom + margin.top)
});

function xloc(d, i) { return i * (barWidth + barPadding); }
function yloc(d) { return maxValue - d; }
function translator(d, i) {
    return "translate(" + xloc(d, i) + "," + yloc(d) + ")";
}

var barGroups = chart.selectAll("g")
    .data(data)
    .enter()
    .append('g')
    .attrs({'transform': translator});
    
barGroups.append('rect')
    .attrs({
        'fill': 'steelblue',
        'width': barWidth,
        'height': function(d) { return d; }
    });

var textTranslator = "translate(" + barWidth / 2 + ", 2)";

barGroups.append('text')
    .attrs({
        'transform': textTranslator,
        'fill': 'white',
        'alignment-baseline': 'before-edge',
        'text-anchor': 'middle',  
    })
    .style('font', '10px sans-serif')
    .text(function (d) { return d; });

var scale = d3.scaleLinear()
            .domain([maxValue, 0])
            .range([0, maxValue]);

var axisGroup = svg.append('g');
var axis = d3.axisLeft().scale(scale);
var axisPadding = 3;
axisGroup.call(axis)
    .attr('transform', "translate(" + (margin.left - axisPadding) + "," + margin.top + ")");

# Loading data

In [3]:
%%d3
var url = "https://gist.githubusercontent.com/d3byex/e5ce6526ba2208014379/raw/8fefb14cc18f0440dc00248f23cbf6aec80dcc13/walking_dead_s5.json";
d3.json(url).then(
    function(data) {
        d3.select('body').append('pre').text(JSON.stringify(data,undefined, 2));
    }
).catch(
    function(err) {
        d3.select('body').append('p').text(err);
    }
);

d3.select('body').append('p').text('Data in D3.js is loaded asynchronously');